# Data Wrangling

The recommendation system that we are building will consider both information of residences from Airbnb Hosts as well as city features. In order to input these features into a recommendation system, we will generate a DataFrame containing one row per Host, that will include the features of the residence as well as the city in which the residence is located.

To simplify the work, our initial scope for the project will be residences in the region of Europe.

The dataset with residences has been obtained from https://public.opendatasoft.com/explore/dataset/airbnb-listings

The datasets with features of cities has been obtained from https://ec.europa.eu/eurostat and https://www.numbeo.com

## Import libraries

In [1]:
import pandas as pd
import numpy as np

## Load Files

This is the Airbnb listings dataset including around 500k residences in several countries and continents:

In [2]:
df = pd.read_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\airbnb-listings.csv', sep=';',low_memory=False)

In [3]:
df.head(1)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,15802051,https://www.airbnb.com/rooms/15802051,20170304065726,2017-03-05,Large double available now.,Double room available,NaN,Double room available,none,NaN,...,9.0,10.0,10.0,NaN,NaN,strict,2.0,0.83,"51.4814235208,-0.113258577959",Host Has Profile Pic


In [4]:
#it contains around 500k listings and 89 features per listing
df.shape

(494954, 89)

In [5]:
#check the features of the dataframe
df.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [6]:
#get the list of unique countries
countries = df.Country.unique()
print(countries)

['United Kingdom' 'United States' 'Spain' 'Australia' 'Canada'
 'Netherlands' 'Germany' 'Greece' 'Belgium' 'Ireland' 'Hong Kong'
 'Switzerland' 'France' 'China' 'Denmark' 'Italy' 'Austria' nan 'Mexico'
 '0' 'Uruguay' 'Vanuatu' 'Vatican City' 'Cuba']


In [7]:
# Europe (262k) and North America (160k) have the most residences in the dataset
# Asia/Oceania (46k) is mainly conformed of residences in Australia (40k) and Hong Kong (6k)
# South America (4) have only residual information
#we will focus on the europe residences in this project
df[['Country']].value_counts()

Country       
United States     134545
United Kingdom     61041
France             56562
Spain              45844
Australia          40693
Italy              33146
Canada             30787
Germany            20576
Denmark            20545
Netherlands        15181
Austria             7893
Belgium             7419
Ireland             6729
Hong Kong           6423
Greece              5127
Switzerland         2381
China                 51
Vatican City           2
Mexico                 2
Cuba                   1
Vanuatu                1
Uruguay                1
0                      1
dtype: int64

In [8]:
#subset the original dataset into regions. we will focus on europe in this pilot project
europe = ['United Kingdom','Spain','Netherlands','Germany','Greece','Belgium','Ireland',
      'Switzerland','France','Denmark','Italy','Austria','Vatican City']
america = ['United States','Canada']
asia = ['China','Hong Kong','Australia','Vanuatu']
latam = ['Mexico','Uruguay','Cuba']

#create region datasets
eu = df[df['Country'].isin(europe)]
na = df[df['Country'].isin(america)]
ap = df[df['Country'].isin(asia)]
sa = df[df['Country'].isin(latam)]

## Europe Region - Subsetting the top 50 cities with more listings

We will further analyze and breakdown the europe listings to extract some features that we will utilize in this pilot project. In particular, we are initially interested in extracting the location of the listing, the characteristics of the listing (nº of bedrooms, price, as well as some information about the host and the ratings given by guests.

In [9]:
#there are 282k listings in europe region
eu.shape

(282446, 89)

In [10]:
#For this pilot project, we will focus on the top 50 cities with more listings
first_50 = eu[['Country','City']].value_counts().head(53)
first_50.head(53)

Country         City              
France          Paris                 54512
United Kingdom  London                47423
Germany         Berlin                20454
Italy           Roma                  18618
Spain           Barcelona             17431
Netherlands     Amsterdam             14869
Denmark         København             13805
Spain           Madrid                13207
United Kingdom  Edinburgh              6130
Austria         Wien                   6121
Italy           Rome                   6064
Ireland         Dublin                 4263
Italy           Venezia                4067
Greece          Athina                 3560
Denmark         Copenhagen             3326
                Frederiksberg          2391
Spain           Palma                  1994
Austria         Vienna                 1670
Italy           Venice                 1528
Switzerland     Genève                 1429
Belgium         Bruxelles              1245
Greece          Athens                 10

**Some cities** in the top 50 **are duplicated** in some way, either by having different names (e.g. in english and local) or by being a big burought within a city (e.g. Amsterdam and Amsterdam-Zuidoost).

Some evident duplicates straight away are: "Rome and Roma", "København and Copenhage", "Geneva and Genéve", "Bruxelles and Brussel", "Venice and Venize" and "Palma and Palma de Mallorca", "London and Greater London", "Amsterdam and Amsterdam-Zuidoost" and "Athens, Athina and Αθήνα".

However, **there may be also boroughts in big cities that do not show and should be added in the city they belong.**

To solve this issue, we will filter country by country the top 15-20 cities with more listings and individually check whether these buroughts should be added to their city or not.

In [11]:
print('Before city name convergence, the first 50 cities group',np.round((first_50.sum()/282444),2)*100,'% of the listings in the dataset')

Before city name convergence, the first 50 cities group 91.0 % of the listings in the dataset


## Convergence of i. cities named in different ways, and ii. buroughts within such cities

Before starting with this task, we will set some criteria to make the convergence:

1. We will first list the 15 cities with more listings per country. If the listing count is still very high at city 15, we will add 5 more and keep repeating until the number is in line with other cities.
2. In relation with the **buroughts/small towns/populations** close or very close to big cities:
   * if they are clearly a burough or very close to the city (inside peripheria) we will add them to the city.
   * if the city is very wide, and therefore has a wider urbanised peripheria, we will set a limit radius and only pick the ones inside that radius. Depending of the city area, this radius will be larger or smaller.
   * If they can be visually split from the city in a map, due to having some unurbanized area between them, we will not consider them as part of the city.

Based on this criteria, we will proceed to identify and make a convergence of city names and buroughts. After doing this, we expect to see a different set of the 50 cities with more listings, and a greater number of listings within the cities that already appeared in the first top 50 list.

### France top cities by nº of residences

Top 15 cities in France are actually all different Paris buroughs, we will converge all of them into Paris.

Acknowledging that the *"Arrondissments"* are the buroughs from Paris, we decided to extend the list of cities to get them all, even though the count was already small enough to stop there in the first 15 cities.

In [12]:
# 1 France cities
eu[eu['Country'] =='France'][['Country','City']].value_counts().head(26)

Country  City                    
France   Paris                       54512
         Paris-19E-Arrondissement      170
         Paris-15E-Arrondissement      148
         Paris-18E-Arrondissement      121
         Paris-20E-Arrondissement      112
         Paris-16E-Arrondissement       94
         Paris-11E-Arrondissement       90
         Paris-10E-Arrondissement       81
         Paris-17E-Arrondissement       77
         Paris-13E-Arrondissement       76
         Paris-14E-Arrondissement       75
         Paris-12E-Arrondissement       70
         Paris-4E-Arrondissement        64
         Boulogne-Billancourt           61
         Paris-3E-Arrondissement        61
         Paris-1ER-Arrondissement       43
         Neuilly-sur-Seine              41
         Paris-2E-Arrondissement        41
         Paris-5E-Arrondissement        40
         PARIS                          32
         Paris-8E-Arrondissement        30
         Issy-les-Moulineaux            29
         Paris-6E-Ar

In [13]:
#create a list with all the duplicates from paris
Paris_duplicates = ['Paris-19E-Arrondissement','Paris-15E-Arrondissement','Paris-18E-Arrondissement','Paris-20E-Arrondissement',
                   'Paris-16E-Arrondissement','Paris-11E-Arrondissement','Paris-10E-Arrondissement','Paris-17E-Arrondissement',
                   'Paris-13E-Arrondissement','Paris-14E-Arrondissement','Paris-12E-Arrondissement','Paris-4E-Arrondissement',
                   'Paris-3E-Arrondissement','Paris-1ER-Arrondissement','Paris-2E-Arrondissement','Paris-5E-Arrondissement',
                    'Paris-8E-Arrondissement','Paris-6E-Arrondissement','Paris-6E-Arrondissement','Paris-7E-Arrondissement',
                   'paris','PARIS']

### Germany top cities by nº of residences

Top 15 cities in Germany are mainly Berlin in different languages and Berlin buroughs.

We converge them directly to Berlin.

In [14]:
# 2 Germany cities
eu[eu['Country'] =='Germany'][['Country','City']].value_counts().head(15)

Country  City                  
Germany  Berlin                    20454
         米特                            8
         berlin                        8
         Berlino                       6
         Berlin, Berlin, DE            5
         柏林                            5
         Berlin, friedrichshain        4
         Berlin-Mitte                  4
         Berlin-Schöneberg             3
         Berlin Neukölln               3
         Berlin-Friedrichshain         3
         Prenzlauer Berg               3
         Berlin-Wilmersdorf            3
         Берлин                        3
         Berlin-Neukölln               2
dtype: int64

In [15]:
Berlin_duplicates = ['berlin','Berlino','Berlin, Berlin, DE','Berlin, friedrichshain','Berlin-Mitte','Berlin-Schöneberg','Berlin Neukölln',
                     'Berlin-Friedrichshain','Prenzlauer Berg','Berlin-Wilmersdorf','Берлин','Berlin-Neukölln']

### Spain top cities by nº of residences

Spain had many residences in several different cities and towns, it is quite an special case because compared to other countries, there are many listings in places such as Palma de Mallorca island, which are actually different towns and villages separate from its main city "Palma de Mallorca". We grouped Palma, Alcudia, Pollensa and Inca.

As there were many residences in different cities, we increased ad-hoc the number of cities to be shown to 40 cities, aiming to see if there were more duplicates, but as commented, mostly the cities were individual cities in the island of Palma de Mallorca, being different towns than the city Palma de Mallorca. Therefore, we did not add them.

In [16]:
# 3 Spain cities
eu[eu['Country'] =='Spain'][['Country','City']].value_counts().head(40)

Country  City                      
Spain    Barcelona                     17431
         Madrid                        13207
         Palma                          1994
         Pollença                        991
         Alcúdia                         862
         Palma de Mallorca               728
         Llucmajor                       363
         Manacor                         272
         Campos                          270
         Can Picafort                    266
         Santanyí                        259
         Sóller                          250
         Muro                            246
         Felanitx                        232
         Inca                            202
         Sa Pobla                        198
         Illes Balears                   192
         Colònia de Sant Jordi           176
         Andratx                         165
         Alcudia                         159
         Calvià                          155
         Capdepera 

In [17]:
Palma_duplicates = ['Palma']
Alcudia_duplicates = ['Alcúdia']
Pollensa_duplicates = ['Pollença','Port de Pollença']
Inca_duplicates = ['Selva','Algaida']

### Italy top cities by nº of residences

Top cities from Italy were mainly Rome, Venice and Lido, which were named differently. Also there was a burough from Roma included.

In [18]:
# 4 Italy cities
eu[eu['Country'] =='Italy'][['Country','City']].value_counts().head(15)

Country  City                
Italy    Roma                    18618
         Rome                     6064
         Venezia                  4067
         Venice                   1528
         Lido                      198
         Lido di Ostia             189
         Trento                    132
         Riva del Garda            111
         Arco                       65
         Municipio Roma X           59
         Mestre                     49
         Rovereto                   43
         Madonna di Campiglio       41
         Pinzolo                    37
         Canazei                    32
dtype: int64

In [19]:
Rome_duplicates = ['Roma','Municipio Roma X']
Venice_duplicates = ['Venezia']
Lido_duplicates = ['Lido di Ostia']

### Greece top cities by nº of residences

Top cities from Greece were mainly Athens, named in different languages and buroughs of Athens.

We added them to Athens city.

In [20]:
# 5 Greek cities
eu[eu['Country'] =='Greece'][['Country','City']].value_counts().head(15)

Country  City                    
Greece   Athina                      3560
         Athens                      1041
         Αθήνα                        339
         Kesariani                      9
         Ampelokipoi                    8
         Zografou                       7
         Dafni                          6
         Kentrikos Tomeas Athinon       6
         Neo Psichiko                   5
         Kallithea                      5
         Athènes                        4
         Nea Smirni                     4
         Vironas                        4
         Kolonaki, Athens               4
         Koukaki                        3
dtype: int64

In [21]:
Athens_duplicates = ['Athina','Αθήνα','Athènes','Kesariani','Kolonaki, Athens','Koukaki','Kallithea','Zografou',
                     'Nea Smirni','Ampelokipoi','Vironas','Neo Psichiko','Kentrikos Tomeas Athinon','Dafni']

### Denmark top cities by nº of residences

Top cities from Denmark were mainly Copenhagen and Frederiksberg. There were also several buroughs from Copenhagen and one from Frederiksberg.

We added them to both cities, respectively.

In [22]:
# 6 Denmark cities
eu[eu['Country'] =='Denmark'][['Country','City']].value_counts().head(15)

Country  City           
Denmark  København          13805
         Copenhagen          3326
         Frederiksberg       2391
         København V          107
         København S           88
         Valby                 88
         København N           81
         Hellerup              69
         Vanløse               60
         København K           59
         Frederiksberg C       51
         København Ø           40
         København NV          28
         Brønshøj              27
         Kastrup               20
dtype: int64

In [23]:
Copenhagen_duplicates =['København','København V','København S','København N','København K','København Ø','København NV',
                     'Valby','Brønshøj','Kastrup','Vanløse','Frederiksberg','Frederiksberg C']

### Austria top cities by nº of residences

Top city in Austria was Vienna written in different languages and some of its buroughs.

We added them to Vienna.

In [24]:
# 7 Austrian cities
eu[eu['Country'] =='Austria'][['Country','City']].value_counts().head(15)

Country  City          
Austria  Wien              6121
         Vienna            1670
         维也纳                 20
         Wien, Wien, AT       8
         wien                 4
         Innere Stadt         3
         Josefstadt           3
         Wieden               3
         Wien 1180            3
         Brigittenau          2
         Neubau               2
         ウィーン                 2
         Wien 1150            2
         Wien 15              2
         Vienne               2
dtype: int64

In [25]:
Vienna_duplicates = ['Wien','维也纳','Wien, Wien, AT','wien','Innere Stadt','Josefstadt','Brigittenau','Neubau','Wieden','Wien 1180','ウィーン','Wien 1150','Wien 15','Vienne']

### Ireland top cities by nº of residences

Top city in Ireland was Dublin and many of its buroughs and very close peripherical towns or populations.

We added them to the city of Dublin.

In [26]:
# 8 Ireland cities
eu[eu['Country'] =='Ireland'][['Country','City']].value_counts().head(15)

Country  City       
Ireland  Dublin         4263
         Ballsbridge     152
         Rathmines       133
         Drumcondra      116
         Ringsend        114
         Ranelagh         96
         Sandymount       88
         Blackrock        74
         Swords           71
         Dublin 8         57
         Malahide         56
         Clontarf         54
         Donnybrook       45
         Sutton           41
         Lucan            39
dtype: int64

In [27]:
Dublin_duplicates = ['Ballsbridge','Rathmines','Drumcondra','Ringsend','Ranelagh','Sandymount','Dublin 8',
                     'Clontarf','Donnybrook','Blackrock','Lucan','Sutton']

### United Kingdom top cities by nº of residences

Top cities from United kingdom were London, Edinburg and Manchester. There were many towns and populations as well as London buroughs.

We added to London populations or towns that were in a radius of around 15km from the city of London.

In [28]:
# 9 United Kingdom cities. We widen to first 20 as there are more cities in this country. We will include anything
#in a radius of 15 km from london center
eu[eu['Country'] =='United Kingdom'][['Country','City']].value_counts().head(22)

Country         City                
United Kingdom  London                  47423
                Edinburgh                6130
                Greater London            850
                Manchester                810
                Richmond                  261
                Twickenham                255
                Croydon                   222
                Londres                   193
                Harrow                    187
                Edgware                   178
                Wembley                   172
                Ilford                    171
                Kingston upon Thames      166
                Bromley                   137
                Hounslow                  122
                Londra                    111
                Enfield                    95
                Isleworth                  84
                Mitcham                    79
                Sutton                     76
                Barking                    

In [29]:
London_duplicates = ['Greater London','Londres','Londra','Croydon','Twickenham','Edgware',
                     'Wembley','Bromley','Isleworth','Mitcham']

### Belgium top cities by nº of residences

Top cities from Belgium were Brussels and Antwerpen written in different ways as well as many buroughs of Brussels.

We added them to their respective cities.

In [30]:
# 10 Belgium cities
eu[eu['Country'] =='Belgium'][['Country','City']].value_counts().head(20)

Country  City                 
Belgium  Bruxelles                1245
         Antwerpen                1026
         Ixelles                  1016
         Saint-Gilles              654
         Brussels                  397
         Schaerbeek                380
         Etterbeek                 348
         Forest                    275
         Uccle                     245
         Brussel                   219
         Antwerp                   193
         Anderlecht                186
         Saint-Josse-ten-Noode     147
         Woluwe-Saint-Pierre        98
         Ville de Bruxelles         97
         Auderghem                  96
         Woluwe-Saint-Lambert       85
         Molenbeek-Saint-Jean       79
         Watermael-Boitsfort        62
         City of Brussels           59
dtype: int64

In [31]:
Brussels_duplicates = ['Brussel','Bruxelles','Ixelles','Saint-Gilles','Schaerbeek','Etterbeek','Forest','Uccle','Anderlecht',
                       'Saint-Josse-ten-Noode','Woluwe-Saint-Pierre','Ville de Bruxelles','Auderghem','Woluwe-Saint-Lambert'
                      ,'Molenbeek-Saint-Jean','Watermael-Boitsfort','City of Brussels']
Antwerp_duplicates = ['Antwerpen']


### Netherlands top cities by nº of residences

Top cities from The Netherlands were Amsterdam written in different ways as well as several buroughs of Amsterdam.

We added them to the city of Amsterdam.

In [32]:
# 11 Netherlands cities
eu[eu['Country'] =='Netherlands'][['Country','City']].value_counts().head(20)

Country      City                        
Netherlands  Amsterdam                       14869
             Amsterdam-Zuidoost                198
             Amsterdam Zuid-Oost                54
             De Pijp                             6
             Amstelveen                          5
             Jordaan                             5
             Watergraafsmeer                     5
             Amsterdam Zuidoost                  4
             Ámsterdam                           4
             Badhoevedorp                        3
             Amsterdam (Zunderdorp)              3
             Diemen                              2
             Zeeburg                             2
             Am                                  1
             Amsterda                            1
             AMSTERDAM                           1
             Amsterdam West                      1
             Amsterdam, Noord-Holland, NL        1
             Amsterdão                  

In [33]:
Amsterdam_duplicates = ['Amsterdam-Zuidoost','Amsterdam Zuid-Oost','De Pijp','Amstelveen','Jordaan','Watergraafsmeer',
                       'Amsterdam Zuidoost','Ámsterdam','Amsterdam (Zunderdorp)','Diemen','Zeeburg','Am','Amsterda','AMSTERDAM'
                       ,'Amsterdam West','Amsterdam, Noord-Holland, NL','Амстердам']

### Switzerland top cities by nº of residences

Top city of Switzerland was Geneva written in different ways, as well as several buroughs of Geneva.

We added them to the city of Geneva.

In [34]:
# 12 Switzerland cities
eu[eu['Country'] =='Switzerland'][['Country','City']].value_counts().head(20)

Country      City              
Switzerland  Genève                1429
             Geneva                 430
             Carouge                 79
             Thônex                  26
             Vernier                 25
             Le Grand-Saconnex       24
             Meyrin                  24
             Chêne-Bourg             22
             Lancy                   21
             Onex                    20
             Chêne-Bougeries         19
             Bellevue                18
             Versoix                 18
             Cologny                 15
             Plan-les-Ouates         14
             Grand-Lancy             14
             Collonge-Bellerive      11
             Veyrier                 10
             Vésenaz                  9
             Grand-Saconnex           9
dtype: int64

In [35]:
Geneva_duplicates = ['Genève','Carouge','Thônex','Vernier','Le Grand-Saconnex','Meyrin','Chêne-Bourg','Lancy','Onex','Chêne-Bougeries'
                    ,'Bellevue','Versoix','Cologny','Plan-les-Ouates','Grand-Lancy','Collonge-Bellerive','Veyrier','Vésenaz','Grand-Saconnex']

### Elimination of duplicates and final top 50 cities list

At this point, we proceeded to apply the necessary replacements in line with the duplicates identified previously. After doing this, we recalculated the top 50 cities with more listings and proceeded to save them in a new dataframe called "eu_clean"

In [36]:
#we create a new DataFrame "eu_clean" to do the replacements of duplicated cities
eu_clean = eu

In [37]:
#replacement of the duplicated cities
eu_clean['City'] = eu_clean['City'].replace(Paris_duplicates,'Paris')
eu_clean['City'] = eu_clean['City'].replace(Berlin_duplicates,'Berlin')
eu_clean['City'] = eu_clean['City'].replace(Palma_duplicates,'Palma de Mallorca')
eu_clean['City'] = eu_clean['City'].replace(Alcudia_duplicates,'Alcudia')
eu_clean['City'] = eu_clean['City'].replace(Pollensa_duplicates,'Pollensa')
eu_clean['City'] = eu_clean['City'].replace(Inca_duplicates,'Inca')
eu_clean['City'] = eu_clean['City'].replace(Rome_duplicates,'Rome')
eu_clean['City'] = eu_clean['City'].replace(Venice_duplicates,'Venice')
eu_clean['City'] = eu_clean['City'].replace(Lido_duplicates,'Lido')
eu_clean['City'] = eu_clean['City'].replace(Athens_duplicates,'Athens')
eu_clean['City'] = eu_clean['City'].replace(Copenhagen_duplicates,'Copenhagen')
eu_clean['City'] = eu_clean['City'].replace(Vienna_duplicates,'Vienna')
eu_clean['City'] = eu_clean['City'].replace(Dublin_duplicates,'Dublin')
eu_clean['City'] = eu_clean['City'].replace(London_duplicates,'London')
eu_clean['City'] = eu_clean['City'].replace(Brussels_duplicates,'Brussels')
eu_clean['City'] = eu_clean['City'].replace(Antwerp_duplicates,'Antwerp')
eu_clean['City'] = eu_clean['City'].replace(Amsterdam_duplicates,'Amsterdam')
eu_clean['City'] = eu_clean['City'].replace(Geneva_duplicates,'Geneva')

C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [38]:
#check again the first new 50 cities with more residences
first_50_bis = eu_clean[['Country','City']].value_counts().head(50)
first_50_bis.head(50)

Country         City                      
France          Paris                         56008
United Kingdom  London                        49704
Italy           Rome                          24741
Germany         Berlin                        20501
Denmark         Copenhagen                    20171
Spain           Barcelona                     17431
Netherlands     Amsterdam                     15163
Spain           Madrid                        13207
Austria         Vienna                         7847
United Kingdom  Edinburgh                      6130
Belgium         Brussels                       5688
Italy           Venice                         5595
Ireland         Dublin                         5272
Greece          Athens                         5005
Spain           Palma de Mallorca              2722
Switzerland     Geneva                         2237
Belgium         Antwerp                        1219
Spain           Pollensa                       1206
                Alcud

In [39]:
#recalculation of how many listings we have in the top 50 cities compared to the original dataset
print('The top 50 cities now include',np.round(first_50_bis.sum()/282444,2)*100,'% of the residences in the Europe region.')

The top 50 cities now include 95.0 % of the residences in the Europe region.


### Final subset of Europe region

The elimination of duplicities increased the number of listings in the previous top 50 from 91% of the original dataset to 95%. There are new cities in the lower braket that previously did not appear.

There may be some cities that are still duplicated or buroughs that should be added, but for the sake of practicity, and being this project a pilot prototype, we will proceed with this list as it is.

In [40]:
first_50_bisdf = pd.DataFrame(first_50_bis)
first_50_bisdf.reset_index(inplace=True)
list_cities = first_50_bisdf['City'].to_list()
list_countries = first_50_bisdf['Country'].to_list()
list_cities

['Paris',
 'London',
 'Rome',
 'Berlin',
 'Copenhagen',
 'Barcelona',
 'Amsterdam',
 'Madrid',
 'Vienna',
 'Edinburgh',
 'Brussels',
 'Venice',
 'Dublin',
 'Athens',
 'Palma de Mallorca',
 'Geneva',
 'Antwerp',
 'Pollensa',
 'Alcudia',
 'Manchester',
 'Inca',
 'Lido',
 'Llucmajor',
 'Manacor',
 'Campos',
 'Can Picafort',
 'Richmond',
 'Santanyí',
 'Sóller',
 'Muro',
 'Felanitx',
 'Sa Pobla',
 'Illes Balears',
 'Harrow',
 'Colònia de Sant Jordi',
 'Ilford',
 'Kingston upon Thames',
 'Andratx',
 'Calvià',
 'Capdepera',
 'Trento',
 'Son Servera',
 'Santa Margalida',
 'Cala Ratjada',
 'Hounslow',
 'Sant Llorenç des Cardassar',
 'Artà',
 'Riva del Garda',
 'Can Pastilla',
 'Valldemossa']

### Final Europe region dataset

We save the final europe region dataset as "eu_final" and will proceed to impute missing values of the columns that will be used in the recommender system.

In [41]:
#filter eu clean dataframe to only consider the cities of "cities list"
eu_final = eu_clean[eu_clean['City'].isin(list_cities)]

In [42]:
eu_final.shape

(267750, 89)

In [43]:
eu_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267750 entries, 0 to 491914
Data columns (total 89 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              267750 non-null  object 
 1   Listing Url                     267750 non-null  object 
 2   Scrape ID                       267750 non-null  object 
 3   Last Scraped                    267750 non-null  object 
 4   Name                            267401 non-null  object 
 5   Summary                         258101 non-null  object 
 6   Space                           186288 non-null  object 
 7   Description                     267675 non-null  object 
 8   Experiences Offered             267750 non-null  object 
 9   Neighborhood Overview           155370 non-null  object 
 10  Notes                           100582 non-null  object 
 11  Transit                         162095 non-null  object 
 12  Access          

In [44]:
#we will check that the ID of the listing is unique for every residence
#There are 2 ID's which are duplicated
eu_final[['ID']].nunique()

ID    267748
dtype: int64

In [45]:
#filter them to see which ID's are repeated
duplicated = eu_final[eu_final.duplicated(['ID'])]
duplicated.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
325397,12512133,https://www.airbnb.com/rooms/12512133,20160709001120,2016-07-09,Spacious Central 4 Bed Apartment,"This light and airy apartment, located near Ge...","The apartment is over two storeys, with 2 bedr...","This light and airy apartment, located near Ge...",none,The apartment is very well located. George Squ...,...,8.0,10.0,10.0,NaN,NaN,strict,1.0,1.0,"55.9436642918,-3.18404774644","Host Has Profile Pic,Is Location Exact"
473754,2832508,https://www.airbnb.com/rooms/2832508,20160709001120,2016-07-09,"Bethlehem way, Edinburgh , Scotland",Whether you're in Edinburgh on business or lei...,The apartment is equipped with complimentary W...,Whether you're in Edinburgh on business or lei...,none,2 bedroom flat close to city centre in the won...,...,10.0,9.0,9.0,NaN,NaN,strict,1.0,1.0,"55.9639969728,-3.16185946779",Host Has Profile Pic


In [46]:
#ID 12512133 appears 2 times in the dataset
eu_final[eu_final['ID'] == '12512133']

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
56034,12512133,https://www.airbnb.com/rooms/12512133,20160709001120,2016-07-09,Spacious Central 4 Bed Apartment,"This light and airy apartment, located near Ge...","The apartment is over two storeys, with 2 bedr...","This light and airy apartment, located near Ge...",none,The apartment is very well located. George Squ...,...,8.0,10.0,10.0,NaN,NaN,strict,1.0,1.0,"55.9436642918,-3.18404774644","Host Has Profile Pic,Is Location Exact"
325397,12512133,https://www.airbnb.com/rooms/12512133,20160709001120,2016-07-09,Spacious Central 4 Bed Apartment,"This light and airy apartment, located near Ge...","The apartment is over two storeys, with 2 bedr...","This light and airy apartment, located near Ge...",none,The apartment is very well located. George Squ...,...,8.0,10.0,10.0,NaN,NaN,strict,1.0,1.0,"55.9436642918,-3.18404774644","Host Has Profile Pic,Is Location Exact"


In [47]:
#ID 2832508 appears 2 times in the dataset
eu_final[eu_final['ID'] == '2832508']

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
230188,2832508,https://www.airbnb.com/rooms/2832508,20160709001120,2016-07-09,"Bethlehem way, Edinburgh , Scotland",Whether you're in Edinburgh on business or lei...,The apartment is equipped with complimentary W...,Whether you're in Edinburgh on business or lei...,none,2 bedroom flat close to city centre in the won...,...,10.0,9.0,9.0,NaN,NaN,strict,1.0,1.0,"55.9639969728,-3.16185946779",Host Has Profile Pic
473754,2832508,https://www.airbnb.com/rooms/2832508,20160709001120,2016-07-09,"Bethlehem way, Edinburgh , Scotland",Whether you're in Edinburgh on business or lei...,The apartment is equipped with complimentary W...,Whether you're in Edinburgh on business or lei...,none,2 bedroom flat close to city centre in the won...,...,10.0,9.0,9.0,NaN,NaN,strict,1.0,1.0,"55.9639969728,-3.16185946779",Host Has Profile Pic


In [48]:
#we will keep the first record only
eu_final = eu_final.drop_duplicates(subset='ID', keep="first")

In [49]:
#we set the index to be ID
eu_final.set_index('ID', inplace=True)

In [50]:
eu_final.head(2)

,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
ID,,,,,,,,,,,,,,,,,,,,,
15802051,https://www.airbnb.com/rooms/15802051,20170304065726,2017-03-05,Large double available now.,Double room available,NaN,Double room available,none,NaN,NaN,...,9.0,10.0,10.0,NaN,NaN,strict,2.0,0.83,"51.4814235208,-0.113258577959",Host Has Profile Pic
14751869,https://www.airbnb.com/rooms/14751869,20170304065726,2017-03-05,Spacious 4BR home near Brixton and Clapham,My home is a large four bedroom space set over...,"This is a bright, tidy, and spacious home in a...",My home is a large four bedroom space set over...,none,Both Brixton and Clapham are very trendy areas...,Please note: our standard check in times are b...,...,10.0,9.0,10.0,NaN,NaN,moderate,1.0,3.73,"51.455615821,-0.122839067385","Host Is Superhost,Host Has Profile Pic,Host Id..."


In [51]:
#check all the columns
#we will split the columns into groups based on their content
#for this prototype project we are interested in house characteristics and host information
eu_final.columns

Index(['Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary', 'Space',
       'Description', 'Experiences Offered', 'Neighborhood Overview', 'Notes',
       'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url',
       'Medium Url', 'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL',
       'Host Name', 'Host Since', 'Host Location', 'Host About',
       'Host Response Time', 'Host Response Rate', 'Host Acceptance Rate',
       'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood',
       'Host Listings Count', 'Host Total Listings Count',
       'Host Verifications', 'Street', 'Neighbourhood',
       'Neighbourhood Cleansed', 'Neighbourhood Group Cleansed', 'City',
       'State', 'Zipcode', 'Market', 'Smart Location', 'Country Code',
       'Country', 'Latitude', 'Longitude', 'Property Type', 'Room Type',
       'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type',
       'Amenities', 'Square Feet', 'Price', 'Weekly Price', 'Monthly Price',
  

## Create different datasets containing features of the same type

This will allow us to easily clean each of them separately, focusing on the features we really want to extract, to later on add them together based on the IDs that are not null in each of the datasets.

* **res_characteristics**: informamtion about the type of residence and number of rooms by type (bathrooms, bedrooms, .. ) and the host characteristics (has profile picture, is verified, requires guest phone number,..)
* **res_rating**: ratings of the residence by previous guests on different subjects, such as cleaningless, communication with host.. 
* **res_price**: daily, weekly and monthly price of the residence, as well as other extra fees (cleaning, deposit,..)


* res_location: information about the country and city of the residence, street, zipcode..
* res_availability: different time periods in which the house is available
* host_rating: some metrics about the host, such as response rate, number of verifications by guests, .. 
* res_text: different text fields describing the host, residence, neighbour, house rules, ..
* res_url: url address to residence and host pictures

For this project we will work with **res_characteristics, res_rating and res_price.**

In [52]:

#info about the residence structure
res_characteristics = eu_final[['Features','Property Type', 'Room Type',
       'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type',
       'Square Feet']]

#info about the residence ratings
res_rating = eu_final[['First Review', 'Last Review','Number of Reviews',
                 'Review Scores Rating','Review Scores Accuracy',
               'Review Scores Cleanliness', 'Review Scores Checkin',
               'Review Scores Communication', 'Review Scores Location',
               'Review Scores Value','Reviews per Month']]

#info about the residence price
res_price = eu_final[['Price', 'Weekly Price', 'Monthly Price',
       'Security Deposit', 'Cleaning Fee', 'Guests Included',
               'Extra People','Cancellation Policy']]

#info about the residence location
res_location = eu_final[['Country Code','Country','City','State','Street',
                   'Zipcode','Market','Smart Location']]

#info about the residence availability
res_availability = eu_final[['Has Availability', 'Availability 30', 'Availability 60',
       'Availability 90', 'Availability 365']]

#info about the host rating
host_rating = eu_final[['Host Response Time', 'Host Response Rate', 'Host Acceptance Rate',
               'Calculated host listings count','Host Listings Count', 'Host Total Listings Count',
       'Host Verifications']]

#info of several text fields
res_text = eu_final[['Host ID','Name', 'Summary', 'Space',
       'Description', 'Experiences Offered', 'Neighborhood Overview', 'Notes',
        'Neighbourhood','Neighbourhood Cleansed', 'Neighbourhood Group Cleansed',
        'City','Host Name', 'Host Since', 'Host Location', 'Host About',
        'Host Neighbourhood','Transit', 'Access', 'Interaction',
        'House Rules','Amenities','Minimum Nights', 'Maximum Nights']]

#info about the residence url and images online
res_url = eu_final[['Listing Url','Thumbnail Url',
        'Medium Url', 'Picture Url', 'XL Picture Url','Host URL','Host Thumbnail Url',
        'Host Picture Url']]


## The listings characteristics (res_characteristics)

The listing characteristics are related to either the strucutre of the residence, through its number of rooms, bathrooms, type of bed, nº of people allowed in the house and property type; and some characteristics about the host and type of booking.

These features are useful to later on cluster residences that are similar. There are many missing values for square feet, therefore we will probably ignore this feature in our recommender system due to being too little populated.

### Imputing missing values for residence characteristics

We will start by imputing missing values of listings based on some criteria.

For the **missing Bathrooms**, we will add 1 as minimum when the value is missing.

For the **missing beds**, if the house states type of bed "Real bed" we will impute 1 bed.

For the **missing Bedrooms**, we will infer it based on the room type and the number of accomodates. In particular:
* if the room type is private room or shared room, we will impute 1 bedroom
* if the room is an entire home/apt, we will impute the following:
 *  if there are less than 2 accommodates, we will impute 1 bedroom
 *  if there are >2 and < 4 accommodates, we will impute 2 bedrooms
 *  if there are >4 and <6 accomodates, we will impute 3 bedrroms
 *  if there are >6 accommodates, we will impute 4 bedrooms

Acknowledging that there are many ways in which this data could be impute, we could also discard the listings with missing features, but with the aim of keeping as many listings as possible, we decided to follow this system rather than dropping such rows.

For the **features**, which represent some host characteristics, we will build a binary dataframe, with columns as categories such as "Has profile pic" and a 0 or a 1 in the rows depending on whether the listing has that feature or not.

#### Imputing Bedrooms missing values

In [53]:
#fill Private rooms and Shared rooms with 1 bedroom in case they are empty
res_characteristics.loc[:,'Bedrooms'] = np.where(((res_characteristics['Bedrooms'].isnull()) & (res_characteristics['Room Type']=='Private room')),1,res_characteristics['Bedrooms'])
res_characteristics.loc[:,'Bedrooms'] = np.where(((res_characteristics['Bedrooms'].isnull()) & (res_characteristics['Room Type']=='Shared room')),1,res_characteristics['Bedrooms'])
#fill Entire Home/Apt with empty bedrooms with 1 if less than 2 accommodates, 2 if >2 and < 4 accomodates, 3 if >4 and <6 accomodates and 4 otherwise
res_characteristics.loc[:,'Bedrooms'] = np.where(((res_characteristics['Bedrooms'].isnull()) & (res_characteristics['Room Type']=='Entire home/apt') & (res_characteristics['Accommodates']<=2)),1,res_characteristics['Bedrooms'])
res_characteristics.loc[:,'Bedrooms'] = np.where(((res_characteristics['Bedrooms'].isnull()) & (res_characteristics['Room Type']=='Entire home/apt') & (res_characteristics['Accommodates']>2) & (res_characteristics['Accommodates']<=4)),2,res_characteristics['Bedrooms'])
res_characteristics.loc[:,'Bedrooms'] = np.where(((res_characteristics['Bedrooms'].isnull()) & (res_characteristics['Room Type']=='Entire home/apt') & (res_characteristics['Accommodates']>4) & (res_characteristics['Accommodates']<=6)),3,res_characteristics['Bedrooms'])
res_characteristics.loc[:,'Bedrooms'] = np.where(((res_characteristics['Bedrooms'].isnull()) & (res_characteristics['Room Type']=='Entire home/apt') & (res_characteristics['Accommodates']>6)),4,res_characteristics['Bedrooms'])

C:\Users\34658\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
C:\Users\34658\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
C:\Users\34658\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [54]:
#there are now no listings without bedrooms
print(res_characteristics['Bedrooms'].isnull().sum())

0


#### Imputing Bathrooms missing values

In [55]:
#impute the bathroom values
res_characteristics.loc[:,'Bathrooms'] = np.where(res_characteristics['Bathrooms'].isnull(), 1, res_characteristics['Bathrooms'])

In [56]:
#check null values for bathrooms
print(res_characteristics['Bathrooms'].isnull().sum())

0


#### Imputing Beds missing values

In [57]:
#impute beds values
res_characteristics.loc[:,'Beds'] = np.where(((res_characteristics['Beds'].isnull()) & (res_characteristics['Bed Type']=='Real Bed')),
                                            1, res_characteristics['Beds'])

In [58]:
#check null values for beds
print(res_characteristics['Beds'].isnull().sum())

0


In [59]:
res_characteristics.head()

,Features,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Square Feet
ID,,,,,,,,,
15802051,Host Has Profile Pic,House,Private room,2.0,1.0,2.0,2.0,Real Bed,NaN
14751869,"Host Is Superhost,Host Has Profile Pic,Host Id...",House,Entire home/apt,8.0,1.5,4.0,4.0,Real Bed,NaN
8901485,"Host Has Profile Pic,Host Identity Verified",Apartment,Private room,2.0,1.0,1.0,1.0,Real Bed,NaN
5832248,Host Has Profile Pic,Apartment,Entire home/apt,6.0,1.0,2.0,4.0,Real Bed,NaN
8049784,Host Has Profile Pic,House,Private room,2.0,1.0,1.0,1.0,Real Bed,NaN


In [60]:
#split the features into a new dataframe "features"
#drop the columns Features and Square feet from the res_characteristics dataframe
features = res_characteristics[['Features']]
res_characteristics = res_characteristics.drop(['Features','Square Feet'],axis=1)

#### Creating the binary features dataframe with host information

The host features bring information about the Host. These features give information about the host, the requirements for guests and about the type of booking of the residence. In particular:

* For the host: if i. has profile picture, ii. is "verified" by previous guests, and iii. is a "super host".
* For the guest requirements: if the guests are requrired to have a: i. profile picutre, ii. phone verification and iii. licencse.
* For the type of booking: if i. location is exact, and ii. if listing is instant bookable.

We will extract these features into a binary table with such categories, being 1 having that categories and 0 otherwise.

In [61]:
features.head()

,Features
ID,
15802051,Host Has Profile Pic
14751869,"Host Is Superhost,Host Has Profile Pic,Host Id..."
8901485,"Host Has Profile Pic,Host Identity Verified"
5832248,Host Has Profile Pic
8049784,Host Has Profile Pic


In [62]:
features['Host pic'] = pd.np.where(features.Features.str.contains("Host Has Profile Pic"),1,0)
features['Host verified'] = pd.np.where(features.Features.str.contains("Host Identity Verified"),1,0)
features['Host superhost'] = pd.np.where(features.Features.str.contains("Host Is Superhost"),1,0)
features['Instant book'] = pd.np.where(features.Features.str.contains("Instant Bookable"),1,0)
features['Req guest phone'] = pd.np.where(features.Features.str.contains("Require Guest Phone Verification"),1,0)
features['Req guest pic'] = pd.np.where(features.Features.str.contains("Require Guest Profile Picture"),1,0)
features['Req license'] = pd.np.where(features.Features.str.contains("Requires License"),1,0)

C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
C:\Users\34658\Anacon

In [63]:
host_features = features.loc[:, features.columns != 'Features']

In [64]:
host_features

,Host pic,Host verified,Host superhost,Instant book,Req guest phone,Req guest pic,Req license
ID,,,,,,,
15802051,1,0,0,0,0,0,0
14751869,1,1,1,1,0,0,0
8901485,1,1,0,0,0,0,0
5832248,1,0,0,0,0,0,0
8049784,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2610162,1,1,0,0,0,0,0
13760682,1,1,0,0,0,0,0
9173969,1,0,0,0,0,0,0


In [65]:
host_features.isnull().sum()

Host pic           0
Host verified      0
Host superhost     0
Instant book       0
Req guest phone    0
Req guest pic      0
Req license        0
dtype: int64

#### Conclussions

We have imputed values for the residence characteristics and generated a new dataframe "res_features" including the information about the host features.

### Listing ratings

The listing ratings show the scores that each listing received from previous guests.

The scores can be about:
* number of reviews: how many guests rated the residence
* rating: a value between 0 and 100
* cleaningless: a value between 0 and 10
* check in: a value between 0 and 10
* communication: a value between 0 and 10
* location: a value between 0 and 10
* value: a value between 0 and 10
* reviews/month: number o reviews divided by number of months the listing has been in the dataset

We will focus on imputing zero values for the residences that have 0 reviews.

We will also calculate a column with the age of the residence, meaning how much time has passed between the first review and the last review.

In [66]:
res_rating.isnull().sum()

First Review                   64673
Last Review                    64735
Number of Reviews                  0
Review Scores Rating           67847
Review Scores Accuracy         68315
Review Scores Cleanliness      68198
Review Scores Checkin          68491
Review Scores Communication    68226
Review Scores Location         68450
Review Scores Value            68506
Reviews per Month              64673
dtype: int64

In [67]:
res_rating.shape

(267748, 11)

In [68]:
res_rating.head()

,First Review,Last Review,Number of Reviews,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month
ID,,,,,,,,,,,
15802051,2016-11-18,2017-01-02,3.0,93.0,9.0,9.0,10.0,9.0,10.0,10.0,0.83
14751869,2016-09-18,2017-02-25,21.0,95.0,10.0,10.0,9.0,10.0,9.0,10.0,3.73
8901485,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5832248,2015-09-19,2016-05-03,2.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.11
8049784,2015-11-03,2016-01-18,3.0,93.0,9.0,9.0,10.0,10.0,9.0,9.0,0.18


In [69]:
#fill with zero values all rows in which Number of reviews is zero
res_rating.loc[res_rating['Number of Reviews'] == 0, ['First Review', 'Last Review','Review Scores Rating','Review Scores Accuracy','Review Scores Cleanliness','Review Scores Checkin','Review Scores Communication','Review Scores Location','Review Scores Value','Reviews per Month']] = 0,0,0,0,0,0,0,0,0,0

C:\Users\34658\Anaconda3\lib\site-packages\pandas\core\indexing.py:1760: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [70]:
res_rating.dtypes

First Review                    object
Last Review                     object
Number of Reviews              float64
Review Scores Rating           float64
Review Scores Accuracy         float64
Review Scores Cleanliness      float64
Review Scores Checkin          float64
Review Scores Communication    float64
Review Scores Location         float64
Review Scores Value            float64
Reviews per Month              float64
dtype: object

In [71]:
res_rating['First Review'] = pd.to_datetime(res_rating['First Review'])
res_rating['Last Review'] = pd.to_datetime(res_rating['Last Review'])

C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [72]:
res_rating['Last Review'] = np.where(res_rating['Last Review'].isnull(),res_rating['First Review'],res_rating['Last Review'])

C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
res_rating['Age'] = res_rating['Last Review'] - res_rating['First Review']

C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
res_rating.head()

,First Review,Last Review,Number of Reviews,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Reviews per Month,Age
ID,,,,,,,,,,,,
15802051,2016-11-18,2017-01-02,3.0,93.0,9.0,9.0,10.0,9.0,10.0,10.0,0.83,45 days
14751869,2016-09-18,2017-02-25,21.0,95.0,10.0,10.0,9.0,10.0,9.0,10.0,3.73,160 days
8901485,1970-01-01,1970-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0 days
5832248,2015-09-19,2016-05-03,2.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.11,227 days
8049784,2015-11-03,2016-01-18,3.0,93.0,9.0,9.0,10.0,10.0,9.0,9.0,0.18,76 days


In [75]:
res_rating.shape

(267748, 12)

In [76]:
#drop all rows with all missing values
res_rating = res_rating.dropna(how='any',subset=['First Review','Last Review','Number of Reviews','Review Scores Rating',
                                                  'Review Scores Accuracy','Review Scores Cleanliness',
                                                  'Review Scores Checkin','Review Scores Communication',
                                                  'Review Scores Location','Review Scores Value','Reviews per Month','Age'])

In [77]:
res_rating.shape

(263547, 12)

In [78]:
res_rating.isnull().sum()

First Review                   0
Last Review                    0
Number of Reviews              0
Review Scores Rating           0
Review Scores Accuracy         0
Review Scores Cleanliness      0
Review Scores Checkin          0
Review Scores Communication    0
Review Scores Location         0
Review Scores Value            0
Reviews per Month              0
Age                            0
dtype: int64

In [79]:
print('After dropping the missing rows we maintain',np.round((263547/267748),2)*100,'% of the res_rating residences')

After dropping the missing rows we maintain 98.0 % of the res_rating residences


### The listings price

The listings price is an important feature to consider, we want to have features that allow us to pair hosts later on. Knowing the price of their residences is a good indicator to match them, as the value of the listing, in monetary terms should be similar so that the trade-off is fair between them.

In big cities, regardless of their residence structure, it is logical that the listings are more expensive than in the country side. However, it should be possible to prioritize the listing price instead of the listing characteristics to make clusters.

In [80]:
res_price.isnull().sum()

Price                    4182
Weekly Price           210869
Monthly Price          218146
Security Deposit       158517
Cleaning Fee           104717
Guests Included             0
Extra People               25
Cancellation Policy         0
dtype: int64

In [81]:
res_price.head()

,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Cancellation Policy
ID,,,,,,,,
15802051,44.0,NaN,NaN,100.0,NaN,1.0,15.0,strict
14751869,139.0,NaN,NaN,400.0,65.0,1.0,0.0,moderate
8901485,70.0,NaN,NaN,NaN,NaN,1.0,0.0,flexible
5832248,70.0,NaN,NaN,NaN,25.0,1.0,12.0,strict
8049784,25.0,NaN,NaN,100.0,NaN,1.0,15.0,moderate


In [82]:
#we recalculated the missing weekly and monthly price based on the daily price of the listings
res_price.loc[:,'Weekly Price'] = np.where(res_price['Weekly Price'].isnull(), res_price['Price']*7, res_price['Weekly Price'])
res_price.loc[:,'Monthly Price'] = np.where(res_price['Monthly Price'].isnull(), res_price['Price']*30, res_price['Monthly Price'])
res_price.loc[:,'Security Deposit'] = np.where(res_price['Security Deposit'].isnull(), 0, res_price['Security Deposit'])
res_price.loc[:,'Cleaning Fee'] = np.where(res_price['Cleaning Fee'].isnull(), 0, res_price['Cleaning Fee'])
res_price.loc[:,'Extra People'] = np.where(res_price['Extra People'].isnull(), 0, res_price['Extra People'])

C:\Users\34658\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [83]:
#we drop all listings without a price, as this is a very important feature to consider
res_price.dropna(subset = ["Price"], inplace=True)

C:\Users\34658\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [84]:
res_price.head()

,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Cancellation Policy
ID,,,,,,,,
15802051,44.0,308.0,1320.0,100.0,0.0,1.0,15.0,strict
14751869,139.0,973.0,4170.0,400.0,65.0,1.0,0.0,moderate
8901485,70.0,490.0,2100.0,0.0,0.0,1.0,0.0,flexible
5832248,70.0,490.0,2100.0,0.0,25.0,1.0,12.0,strict
8049784,25.0,175.0,750.0,100.0,0.0,1.0,15.0,moderate


In [85]:
res_price.isnull().sum()

Price                  0
Weekly Price           0
Monthly Price          0
Security Deposit       0
Cleaning Fee           0
Guests Included        0
Extra People           0
Cancellation Policy    0
dtype: int64

## Join of the different datasets

At this point we have cleaned 3 different datasets:

* res_characteristics
* host features
* res_ratings
* res_price

We will proceed to merge them into a single dataset and add to this dataset the city name, country name and name of the residence:

In [86]:
res_location = res_location[['Country','City']]
res_text = res_text[['Host ID','Name','Summary','Description']]

In [87]:
#res_rating is the dataframe with less values, we will left join on this
res_characteristics.shape, host_features.shape, res_rating.shape, res_price.shape, res_location.shape, res_text.shape

((267748, 7), (267748, 7), (263547, 12), (263566, 8), (267748, 2), (267748, 4))

In [88]:
dataset = pd.concat([res_text,res_location,res_characteristics, host_features,res_rating,res_price], axis=1, join="inner")

In [89]:
dataset.isnull().sum()

Host ID                           0
Name                            324
Summary                        9377
Description                      71
Country                           0
City                              0
Property Type                     1
Room Type                         0
Accommodates                      0
Bathrooms                         0
Bedrooms                          0
Beds                              0
Bed Type                          0
Host pic                          0
Host verified                     0
Host superhost                    0
Instant book                      0
Req guest phone                   0
Req guest pic                     0
Req license                       0
First Review                      0
Last Review                       0
Number of Reviews                 0
Review Scores Rating              0
Review Scores Accuracy            0
Review Scores Cleanliness         0
Review Scores Checkin             0
Review Scores Communication 

In [90]:
dataset['Name'].fillna('Missing Name', inplace=True)

In [91]:
pd.options.display.max_colwidth = 400

In [92]:
print(dataset[dataset['Property Type'].isnull()]['Description'])

ID
7211683    My place is close to Edinburgh city centre, around 10 minutes on the bus which are very frequent or just a short taxi journey. It is also only a 35min journey from the airport on the bus (or around £15-20 in a taxi). My place is good for couples, solo adventurers, and business travelers. The room has a king sized bed, private en-suite and TV. I can provide a basic breakfast of cereal, toast, t...
Name: Description, dtype: object


In [93]:
dataset['Property Type'].fillna('Private room', inplace=True)

In [94]:
dataset['Description'].fillna(dataset['Summary'],inplace=True)
dataset['Summary'].fillna(dataset['Description'],inplace=True)

In [95]:
dataset.isnull().sum()

Host ID                         0
Name                            0
Summary                        66
Description                    66
Country                         0
City                            0
Property Type                   0
Room Type                       0
Accommodates                    0
Bathrooms                       0
Bedrooms                        0
Beds                            0
Bed Type                        0
Host pic                        0
Host verified                   0
Host superhost                  0
Instant book                    0
Req guest phone                 0
Req guest pic                   0
Req license                     0
First Review                    0
Last Review                     0
Number of Reviews               0
Review Scores Rating            0
Review Scores Accuracy          0
Review Scores Cleanliness       0
Review Scores Checkin           0
Review Scores Communication     0
Review Scores Location          0
Review Scores 

In [96]:
dataset['Description'].fillna('Missing Description', inplace=True)
dataset['Summary'].fillna('Missing Summary', inplace=True)

In [97]:
dataset.dtypes

Host ID                                  int64
Name                                    object
Summary                                 object
Description                             object
Country                                 object
City                                    object
Property Type                           object
Room Type                               object
Accommodates                           float64
Bathrooms                              float64
Bedrooms                               float64
Beds                                   float64
Bed Type                                object
Host pic                                 int32
Host verified                            int32
Host superhost                           int32
Instant book                             int32
Req guest phone                          int32
Req guest pic                            int32
Req license                              int32
First Review                    datetime64[ns]
Last Review  

In [98]:
dataset[['Name','Summary','Description','Country','City']] = dataset[['Name','Summary','Description','Country','City']].astype('string')
dataset[['Property Type','Room Type','Cancellation Policy']] = dataset[['Property Type','Room Type','Cancellation Policy']].astype('category')

In [99]:
dataset.dtypes

Host ID                                  int64
Name                                    string
Summary                                 string
Description                             string
Country                                 string
City                                    string
Property Type                         category
Room Type                             category
Accommodates                           float64
Bathrooms                              float64
Bedrooms                               float64
Beds                                   float64
Bed Type                                object
Host pic                                 int32
Host verified                            int32
Host superhost                           int32
Instant book                             int32
Req guest phone                          int32
Req guest pic                            int32
Req license                              int32
First Review                    datetime64[ns]
Last Review  

In [100]:
dataset.head()

,Host ID,Name,Summary,Description,Country,City,Property Type,Room Type,Accommodates,Bathrooms,...,Reviews per Month,Age,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Cancellation Policy
ID,,,,,,,,,,,,,,,,,,,,,
15802051,46296095,Large double available now.,Double room available,Double room available,United Kingdom,London,House,Private room,2.0,1.0,...,0.83,45 days,44.0,308.0,1320.0,100.0,0.0,1.0,15.0,strict
14751869,11850512,Spacious 4BR home near Brixton and Clapham,"My home is a large four bedroom space set over three floors. There is a huge common area with an open plan living room / dining room / kitchen, and the top floor has charming slanted roofs and skylight windows. This space is perfect for a large group of friends or a family who wants all the comforts of home while still being accessible to London.","My home is a large four bedroom space set over three floors. There is a huge common area with an open plan living room / dining room / kitchen, and the top floor has charming slanted roofs and skylight windows. This space is perfect for a large group of friends or a family who wants all the comforts of home while still being accessible to London. This is a bright, tidy, and spacious home in ...",United Kingdom,London,House,Entire home/apt,8.0,1.5,...,3.73,160 days,139.0,973.0,4170.0,400.0,65.0,1.0,0.0,moderate
8901485,12594665,quirky room with a view,Quirky and well decorated bedroom in waterloo area with views of london eye. compact multi purpose room with movable sofa and fold down bed,"Quirky and well decorated bedroom in waterloo area with views of london eye. compact multi purpose room with movable sofa and fold down bed Your room is a living area that converts into a bedroom at night, with aditional shower room, kitchen and toilet room, long balcony with chairs and table. whole house apart from secondary bedroom where host stays. Host will be avalible up until 2pm and aft...",United Kingdom,London,Apartment,Private room,2.0,1.0,...,0.00,0 days,70.0,490.0,2100.0,0.0,0.0,1.0,0.0,flexible
5832248,30273453,Lovely 2r flat in the ❤ of Brixton,Perfectly located flat in the up and coming Brixton. Many popular restaurants and pubs around. Big apartment in a quiet safe street. Great transport links! All amenities you will need. Current available dates in mid September!,"Perfectly located flat in the up and coming Brixton. Many popular restaurants and pubs around. Big apartment in a quiet safe street. Great transport links! All amenities you will need. Current available dates in mid September! The apartment is big, spacy and very bright with direct sunlight. It is located on a well maintained quiet street, just off the main Effra Road. You’ll have the beautifu...",United Kingdom,London,Apartment,Entire home/apt,6.0,1.0,...,0.11,227 days,70.0,490.0,2100.0,0.0,25.0,1.0,12.0,strict
8049784,33670376,Lovely bedroom in a cosy house,Very nice double room in a clean and friendly house. The house is located on a quiet road but just a few steps from shops and public transport. You can also enjoy a big garden to relax after a long day exploring!,"Very nice double room in a clean and friendly house. The house is located on a quiet road but just a few steps from shops and public transport. You can also enjoy a big garden to relax after a long day exploring! All you need in the kitchen for a delicious breakfast! The whole house - sitting room, kitchen, bathroom and garden. As often as they want to. Safety, close shops and very good trans...",United Kingdom,London,House,Private room,2.0,1.0,...,0.18,76 days,25.0,175.0,750.0,100.0,0.0,1.0,15.0,moderate


In [101]:
dataset.isnull().sum()

Host ID                        0
Name                           0
Summary                        0
Description                    0
Country                        0
City                           0
Property Type                  0
Room Type                      0
Accommodates                   0
Bathrooms                      0
Bedrooms                       0
Beds                           0
Bed Type                       0
Host pic                       0
Host verified                  0
Host superhost                 0
Instant book                   0
Req guest phone                0
Req guest pic                  0
Req license                    0
First Review                   0
Last Review                    0
Number of Reviews              0
Review Scores Rating           0
Review Scores Accuracy         0
Review Scores Cleanliness      0
Review Scores Checkin          0
Review Scores Communication    0
Review Scores Location         0
Review Scores Value            0
Reviews pe

In [102]:
dataset.shape

(259415, 40)

In [103]:
print('Before city name convergence, the first 50 cities included',np.round((first_50.sum()/282444),2)*100,'% of the listings in the dataset')
print('The top 50 cities after convergence included',np.round(first_50_bis.sum()/282444,2)*100,'% of the residences in the Europe region.')
print('After cleaning the dataset, the top 50 cities include',np.round(259415/282444,2)*100,'% of the residences in the Europe region.')

Before city name convergence, the first 50 cities included 91.0 % of the listings in the dataset
The top 50 cities after convergence included 95.0 % of the residences in the Europe region.
After cleaning the dataset, the top 50 cities include 92.0 % of the residences in the Europe region.


In [104]:
dataset[['Country','City']].value_counts()

Country         City                      
France          Paris                         54923
United Kingdom  London                        48676
Italy           Rome                          24416
Germany         Berlin                        20114
Spain           Barcelona                     17154
Denmark         Copenhagen                    16024
Netherlands     Amsterdam                     14969
Spain           Madrid                        13031
Austria         Vienna                         7738
United Kingdom  Edinburgh                      6007
Belgium         Brussels                       5591
Italy           Venice                         5552
Ireland         Dublin                         5199
Greece          Athens                         4952
Spain           Palma de Mallorca              2693
Switzerland     Geneva                         2198
Belgium         Antwerp                        1200
Spain           Pollensa                       1183
                Alcud

In [105]:
dataset.loc[:,'City'] = np.where(((dataset['Country']=='United Kingdom') & (dataset['City']=='Dublin')),'London',dataset['City'])
dataset.loc[:,'Country'] = np.where(((dataset['Country']=='Switzerland') & (dataset['City']=='Paris')),'France',dataset['Country'])

In [106]:
dataset[['country_city']] = dataset["Country"] +'_' + dataset["City"]

In [107]:
dataset[['Country','City']].value_counts()

Country         City                      
France          Paris                         54924
United Kingdom  London                        48752
Italy           Rome                          24416
Germany         Berlin                        20114
Spain           Barcelona                     17154
Denmark         Copenhagen                    16024
Netherlands     Amsterdam                     14969
Spain           Madrid                        13031
Austria         Vienna                         7738
United Kingdom  Edinburgh                      6007
Belgium         Brussels                       5591
Italy           Venice                         5552
Ireland         Dublin                         5199
Greece          Athens                         4952
Spain           Palma de Mallorca              2693
Switzerland     Geneva                         2198
Belgium         Antwerp                        1200
Spain           Pollensa                       1183
                Alcud

In [108]:
dataset.dtypes

Host ID                                  int64
Name                                    string
Summary                                 string
Description                             string
Country                                 object
City                                    object
Property Type                         category
Room Type                             category
Accommodates                           float64
Bathrooms                              float64
Bedrooms                               float64
Beds                                   float64
Bed Type                                object
Host pic                                 int32
Host verified                            int32
Host superhost                           int32
Instant book                             int32
Req guest phone                          int32
Req guest pic                            int32
Req license                              int32
First Review                    datetime64[ns]
Last Review  

In [109]:
# save the files into csv for future data wrangling
dataset.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\dataset.csv')

## City features - external datasets

The aim of this project is to make recommendations to hosts about cities and/or residences that they may like, based on their own city and/or residence.

The Airbnb dataset contains information about hosts and residences, but does not contain features about the cities in which the residences are located. Therefore, it is important to add city features to the recommender system.

In line with this idea, we collected several features from cities. The data was obtained from https://ec.europa.eu/eurostat and https://www.numbeo.com/. **Eurostat** is a european database that provides detailed statistical information about european countries. **Numbeo** is a website service that compares numerous countries and cities around the world in terms of purchasing power, crime rates, pollution rates, etc.

In this prototype, we have leveraged on both sources, sometimes calculating our own metrics from the raw datasets. In our research work, we gathered city-features about a number of different "spectrums":

* **Economic spectrum**: To compare cities based on the cost of living, price of groceries, rent, restaurants, or local purchasing power.
   *As a host, it is relevant to be recommended cities that have similar cost of living and purchasing power than their own city.*


* **Social spectrum**: To compare cities based on how locals think that different individuals would enjoy (or not) living in their city considering their age, gender, race or family situation (e.g. Homosexuals, young families with children, elderly, ethnic minorities, etc.).
   *As a host, it is relevant to be recommended cities that have similar social culture than their own city.*


* **Environmental sprectrum**: To compare cities based on how locals perceive the overall air pollution, noise pollution and cleanliness of their city.
   *As a host, it is relevant to be recommended cities that have similar pollution and cleanliness levels than their own city.*


* **Safety spectrum**: To compare cities based on their crime and safety metrics.
   *As a host, it is relevant to be recommended cities that have similar crime and safety levels than their own cities.*


* **City Amenities spectrum**: To compare cities based on how locals perceive the quality and quantity of different services in their city, such as public transport, sport facilities, green areas, etc.
   *As a host, it is relevant to be recommended cities that have similar level of amenities than their own city.*


* **Population spectrum**: To compare cities based on their population number, the number of people within each age group, and the number of males and females within each population age group.
   *As a host, it is relevant to be recommended cities that have similar population number and age groups than their own city.*


As a result of this research work, we have included all the information into a single excel file, maintaining the original city names of the Airbnb dataset.

In [110]:
external_datasets = pd.ExcelFile(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\cities\external_datasets.xlsx')

In [111]:
economic = pd.read_excel(external_datasets,'purchase power rebased')
crime = pd.read_excel(external_datasets,'crime index')
pollution = pd.read_excel(external_datasets,'pollution normalized')
amenities = pd.read_excel(external_datasets,'amenities normalized')
hostility = pd.read_excel(external_datasets,'hostility normalized')
tot_pop = pd.read_excel(external_datasets,'population')
female_pop = pd.read_excel(external_datasets,'population female')
male_pop = pd.read_excel(external_datasets,'population male')
female_rate = pd.read_excel(external_datasets,'ratio female')
male_rate = pd.read_excel(external_datasets,'ratio male')

### Economic spectrum

The economic spectrum is composed of the following metrics, and should be interpreted as follows. Note the description above is from https://www.numbeo.com/cost-of-living/cpi_explained.jsp.

*Remark: We have modified the description, as the base in numbeo for the index is the city of New York, and we have used the average of the 15 cities for each metric as Index.*

* **Cost of Living Index (Excl. Rent)** is a relative indicator of consumer goods prices, including groceries, restaurants, transportation and utilities. Cost of Living Index does not include accommodation expenses such as rent or mortgage. *If a city has a Cost of Living Index of 120, it means Numbeo has estimated it is 20% more expensive than the average of the 15 cities (excluding rent).*

* **Rent Index** is an estimation of prices of renting apartments in the city compared to the average of the 15 cities. *If Rent index is 80, Numbeo has estimated that price of rents in that city is on average 20% less than the average price of the 15 cities.*

* **Groceries Index** is an estimation of grocery prices in the city compared to the average price of the 15 cities. To calculate this section, Numbeo uses weights of items in the "Markets" section for each city.

* **Restaurants Index** is a comparison of prices of meals and drinks in restaurants and bars compared to the average of the 15 cities.

* **Cost of Living Plus Rent Index** is an estimation of consumer goods prices including rent comparing to the average of the 15 cities.

* **Local Purchasing Power** shows relative purchasing power in buying goods and services in a given city for the average net salary in that city. *If domestic purchasing power is 40, this means that the inhabitants of that city with an average salary can afford to buy on an average 60% less goods and services than the average of the 15 cities resident with an average salary.*

In [112]:
economic.head()

,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,Vienna,87.88,76.90,84.28,90.99,81.36,114.38
1,Brussels,96.75,76.97,90.27,97.28,100.70,109.65
2,Antwerp,96.20,61.54,84.85,95.43,107.67,104.97
3,Copenhagen,120.53,122.39,121.14,112.59,134.21,105.80
4,Paris,114.86,117.31,115.66,133.63,104.42,93.94


### Crime spectrum

The crime spectrum is composed of the following metrics, and should be interpreted as follows. For detailed information about the methodology used go to https://www.numbeo.com/crime/indices_explained.jsp.


* **Crime Index** is an estimation of overall level of crime in a given city or a country. We consider. crime levels lower than 20 as very low, crime levels between 20 and 40 as being low, crime levels between 40 and 60 as being moderate, crime levels between 60 and 80 as being high and finally crime levels higher than 80 as being very high.

* **Safety index** is, on the other way, quite opposite of crime index. If the city has a high safety index, it is considered very safe.


In [113]:
crime.head()

,City,Crime Index,Safety Index
0,Manchester,56.29,43.71
1,Paris,54.05,45.95
2,London,52.96,47.04
3,Athens,52.78,47.22
4,Rome,51.99,48.01


### Environmental spectrum

The environmental spectrum is composed of the following metrics, and should be interpreted as follows:


* **air_score_norm**: measures where the city sits in relation with others in terms of the satisfaction with the quality of the air by locals of the city. Values are normalized between 0-1.
* **noise_score_norm**: measures where the city sits in relation with others in terms of the satisfaction with the level of noise by locals of the city. Values are normalized between 0-1.
* **clean_score_norm**: measures where the city sits in relation with others in terms of the satisfaction with the level of cleanliness by locals of the city. Values are normalized between 0-1.

***Methodology to normalize the values of the surveys:***

Each survery has 4 possible answers, i. very satisfied, ii. rather satisfied, iii. rather unsatisfied, and iv. not at all satisfied.

step 1. we redistributed the "don't know/no answer" betwwen the other options, based on their weight.

step 2. we calculated the raw score for that city, by adding the % of votes of groups i and ii, and discounting the groups iii and iv. Also, we weighted more the very positive and very negative votes, by adding a +1.75 multiplier for "very satisfied" and "not at all satisfied".

As an example, if group i. had 50% of voters, group ii. 20, group iii. 10 and group iv. 20, the score formula would be as follows:

* city score = [(50 x 1.75) + 20] - [10 - (20 x 1.75)] = 62.5

step 3. we normalized the scores of the cities, so that the minimum score value would be 0 and the maximum 1, through the following formula. e.g. if the maximum value was 90 and the minimum value was 15, it would be calculated as follows:

* city normalized score = [(62.5 - 15) / (90 - 15)] = 0.633

Through this process, we calculated normalized scores for each feature.

* **Numbeo Pollution Indexes**:

In relation with numbeo's indexes, find detailed information about the methodology utilized in the link enclosed https://www.numbeo.com/pollution/indices_explained.jsp:

* **Pollution Index**:Pollution Index is an estimation of the overall pollution in the city. The biggest weight is given to air pollution, than to water pollution/accessibility, two main pollution factors. Small weight is given to other pollution types.

* **Pollution Exp Scale** is using an exponential scale to show very high numbers for very polluted cities, and very low numbers for unpolluted cities. Therefore to calculate formula it uses the exponential function to calculate the index.:

In [114]:
pollution.head()

,City,air_score_norm,noise_score_norm,clean_score_norm,Pollution Index,Exp Pollution Index
0,Brussels,0.388329,0.408581,0.422918,62.13,110.17
1,Antwerp,0.594383,0.774693,0.836928,61.14,110.83
2,Copenhagen,0.712932,0.726070,0.801142,21.06,33.38
3,Berlin,0.726065,0.549622,0.523524,39.41,65.98
4,Dublin,0.963148,1.000000,0.678932,39.88,66.51


### Amenties spectrum

The amenties spectrum is composed of the following metrics, and should be interpreted as follows:

* **norm_score_pub_transport**: measures where the city sits in relation with others in terms of how satisfied are locals with Public transport in the city, for example bus, tram or metro. Values are normalized between 0-1.
* **norm_score_green_places**: measures where the city sits in relation with others in terms of how satisfied are locals with Green spaces such as public parcs or gardens. Values are normalized between 0-1.
* **norm_score_sports**: measures where the city sits in relation with others in terms of how satisfied are locals with Sports facilities such as sport fields and indoor sport halls in the city. Values are normalized between 0-1.
* **norm_score_cultural**: measures where the city sits in relation with others in terms of how satisfied are locals with Cultural facilities such as concert halls, theatres, museums and libraries in the city. Values are normalized between 0-1.
* **norm_score_public_spaces**: measures where the city sits in relation with others in terms of how satisfied are locals with Public spaces in this city such as markets, squares, pedestrian areas. Values are normalized between 0-1.


***Methodology to normalize the values of the surveys:***

Each survery has 4 possible answers, i. very satisfied, ii. rather satisfied, iii. rather unsatisfied, and iv. not at all satisfied.

step 1. we redistributed the "don't know/no answer" betwwen the other options, based on their weight.

step 2. we calculated the raw score for that city, by adding the % of votes of groups i and ii, and discounting the groups iii and iv. Also, we weighted more the very positive and very negative votes, by adding a +1.75 multiplier for "very satisfied" and "not at all satisfied".

As an example, if group i. had 50% of voters, group ii. 20, group iii. 10 and group iv. 20, the score formula would be as follows:

* city score = [(50 x 1.75) + 20] - [10 - (20 x 1.75)] = 62.5

step 3. we normalized the scores of the cities, so that the minimum score value would be 0 and the maximum 1, through the following formula. e.g. if the maximum value was 90 and the minimum value was 15, it would be calculated as follows:

* city normalized score = [(62.5 - 15) / (90 - 15)] = 0.633

Through this process, we calculated normalized scores for each feature.

In [115]:
amenities.head()

,City,norm_score_pub_transport,norm_score_green_places,norm_score_sports,norm_score_cultural,norm_score_public_spaces
0,Brussels,0.583663,0.811268,0.707202,0.537342,0.736297
1,Antwerp,0.602647,0.856023,1.000000,0.749101,0.918953
2,Copenhagen,0.732794,0.970956,0.744960,0.796250,0.924146
3,Berlin,0.821741,0.867674,0.603141,0.731351,0.767469
4,Dublin,0.652075,0.919839,0.831729,0.648258,0.751763


### Hostility spectrum

The hostility spectrum is composed of the following metrics, and should be interpreted as follows:

* **score_norm_general**: measures where the city sits in relation with others in terms of how locals believe that, for people in general, their city is a good (or bad) place to live. Values are normalized between 0-1.
* **score_norm_ethnic**: measures where the city sits in relation with others in terms of how locals believe that, for racial and ethnic minorities, their city is a good (or bad) place to live. Values are normalized between 0-1.
* **score_norm_gay**: measures where the city sits in relation with others in terms of how locals believe that, for gay or lesbian people, their city is a good (or bad) place to live. Values are normalized between 0-1.
* **score_norm_immigrants**: measures where the city sits in relation with others in terms of how locals believe that, for immigrants from other countries, their city is a good (or bad) place to live. Values are normalized between 0-1.
* **score_norm_young_families**: measures where the city sits in relation with others in terms of how locals believe that, for young families with children, their city is a good (or bad) place to live. Values are normalized between 0-1.
* **score_norm_elderly**: measures where the city sits in relation with others in terms of how locals believe that, for elderly people, their city is a good (or bad) place to live. Values are normalized between 0-1.

***Methodology to normalize the values of the surveys:***

Each survery has 2 possible answers, i. a good place to live and ii. a bad place to live.

step 1. we redistributed the "don't know/no answer" between the other options, based on their weight.

step 2. we calculated the raw score for that city, by adding the % of votes of groups i and discounting the number of votes of group ii.

As an example, if group i. had 90% of voters, group ii. 10%, the score formula would be as follows:

* city score = [90 -10] = 80

step 3. we normalized the scores of the cities, so that the minimum score value would be 0 and the maximum 1, through the following formula. e.g. if the maximum value was 95 and the minimum value was 50, it would be calculated as follows:

* city normalized score = [(80 - 50) / (95 - 50)] = 0.666

Through this process, we calculated normalized scores for each feature.

In [116]:
hostility.head()

,City,score_norm_general,score_norm_ethnic,score_norm_gay,score_norm_immigrants,score_norm_young_families,score_norm_elderly
0,Brussels,0.725835,0.545045,0.061683,0.534006,0.434955,0.633666
1,Antwerp,0.967707,0.482883,0.822269,0.862052,0.439450,0.701256
2,Copenhagen,0.854130,0.625225,0.976477,0.653689,0.970122,0.803943
3,Berlin,0.905536,0.648348,0.795609,0.763278,0.652565,0.654246
4,Dublin,0.938489,0.971171,0.912703,0.977169,0.585933,0.734185


### Population spectrum

The population spectrum is composed of the following files, that contain:

* **tot_pop**: total population per city (male+female), in brakets of 5 years (i.e. from 20-24 years, from 25-29..) as well as the weight of such age groups in the total population.
   *As a host, it is relevant to be recommended cities with similar number of people as their own city.*
* **female_pop**: idem, but only considering the female population per city.
   *As a host, it is relevant to be recommended cities with similar number of females in their age group.*
* **male_pop**: idem, but only considering the male population per city.
   *As a host, it is relevant to be recommended cities with similar number of males in their age group.*
* **female_rate**: the ratio of females:males for each age group in the city.
   *As a host, it is relevant to be recommended cities with similar female:male ratios as in their own city.*
* **male_rate**: the ratio of males:females for each age group in the city.
   *As a host, it is relevant to be recommended cities with similar male:female ratios as in their own city.*

In [117]:
tot_pop.head()

,good,total population,-5,5-9,10-14,15-19,20-24,25-29,30-34,35-39,...,(%)80-84,(%)85-89,(%)90,(%)20-29,(%)30-39,(%)40-49,(%)50-59,(%)60-69,(%)70-79,(%)80-89
0,Amsterdam,3302548,170728,175585,180820,192348,201080,240026,238036,216988,...,2.17,1.27,0.66,13.36,14.48,13.78,13.15,14.30,8.20,3.44
1,Antwerp,1059946,61399,64548,61894,56939,61361,72446,72308,71005,...,2.87,1.96,1.06,12.62,13.65,13.52,12.55,13.25,7.90,4.83
2,Athens,3557383,161215,169483,175997,174513,178558,183283,197764,255229,...,3.31,2.05,1.26,10.17,10.71,12.73,15.86,14.45,9.04,5.36
3,Barcelona,5635100,251234,289202,312241,294346,287869,319129,338465,392893,...,2.63,2.08,1.19,10.77,11.67,12.98,16.77,14.00,8.39,4.71
4,Berlin,5342887,268420,251177,232401,220805,262874,341293,437408,403817,...,3.71,1.68,0.85,11.31,14.58,15.75,12.54,15.42,8.76,5.39


In [118]:
female_pop.head()

,City,total female population,-5,5-9,10-14,15-19,20-24,25-29,30-34,35-39,...,(%)80-84,(%)85-89,(%)90,(%)20-29,(%)30-39,(%)40-49,(%)50-59,(%)60-69,(%)70-79,(%)80-89
0,Amsterdam,1667602,83040,85735,88294,94097,101083,121002,119213,108561,...,2.42,1.57,0.93,13.32,13.66,13.14,14.09,11.34,8.49,3.99
1,Antwerp,534038,30012,31548,30339,27544,29980,36215,36171,35067,...,3.26,2.42,1.46,12.39,13.34,12.21,12.92,11.24,8.39,5.68
2,Athens,1873261,78775,82816,85131,81697,85211,92906,102913,129055,...,3.81,2.44,1.47,9.51,12.38,15.62,14.94,12.62,9.68,6.25
3,Barcelona,2901398,122134,139871,150390,141451,139508,159500,171677,201351,...,3.05,2.60,1.68,10.31,12.86,16.31,13.86,11.19,9.04,5.65
4,Berlin,2712610,130552,122286,112773,106964,129246,170430,216691,198390,...,4.31,2.09,1.23,11.04,15.30,12.12,15.03,11.99,9.45,6.40


In [119]:
male_pop.head()

,City,total male population,-5,5-9,10-14,15-19,20-24,25-29,30-34,35-39,...,(%)35-39,(%)40-44,(%)45-49,(%)50-54,(%)55-59,(%)60-64,(%)65-69,(%)70-74,(%)75-79,(%)80-84
0,Amsterdam,1634946,87688,89850,92526,98251,99997,119024,118823,108427,...,6.63,6.21,6.96,7.38,7.14,6.07,5.24,4.84,3.07,1.92
1,Antwerp,525908,31387,33000,31555,29395,31381,36231,36137,35938,...,6.83,6.43,6.47,6.75,6.83,6.06,5.06,4.32,3.08,2.48
2,Athens,1684122,82440,86667,90866,92816,93347,90377,94851,126174,...,7.49,8.30,7.82,7.50,6.42,5.93,5.32,4.93,3.40,2.76
3,Barcelona,2733702,129100,149331,161851,152895,148361,159629,166788,191542,...,7.01,8.70,8.57,7.58,6.57,5.66,4.75,4.37,3.32,2.18
4,Berlin,2630277,137868,128891,119628,113841,133628,170863,220717,205427,...,7.81,6.66,6.30,7.77,8.04,6.29,5.29,3.69,4.35,3.09


In [120]:
female_rate.head()

,City,f:m_-5,f:m_5-9,f:m_10-14,f:m_15-19,f:m_20-24,f:m_25-29,f:m_30-34,f:m_35-39,f:m_40-44,...,f:m_80-84,f:m_85-89,f:m_90,f:m20-29,f:m_30-39,f:m_40-49,f:m_50-59,f:m_60-69,f:m_70-79,f:m_80-89
0,Amsterdam,94.70,95.42,95.43,95.77,101.09,101.66,100.33,100.12,102.19,...,128.84,166.14,245.45,101.40,100.23,101.78,98.91,102.22,109.45,141.31
1,Antwerp,95.62,95.60,96.15,93.70,95.54,99.96,100.09,97.58,96.40,...,133.30,164.91,227.96,97.90,98.84,96.20,96.62,102.69,115.13,145.15
2,Athens,95.55,95.56,93.69,88.02,91.28,102.80,108.50,102.28,104.41,...,153.88,169.27,157.35,96.95,104.95,107.80,119.33,124.72,129.31,159.54
3,Barcelona,94.60,93.67,92.92,92.52,94.03,99.92,102.93,105.12,101.00,...,148.43,181.18,262.41,97.08,104.10,100.25,103.90,114.04,124.79,161.89
4,Berlin,94.69,94.88,94.27,93.96,96.72,99.75,98.18,96.57,96.22,...,143.85,172.45,275.81,98.42,97.40,96.37,98.01,106.84,121.28,152.08


In [121]:
male_rate.head()

,City,m:f_-5,m:f_5-9,m:f_10-14,m:f_15-19,m:f_20-24,m:f_25-29,m:f_30-34,m:f_35-39,m:f_40-44,...,m:f_80-84,m:f_85-89,m:f_90,f:m20-29,f:m_30-39,f:m_40-49,f:m_50-59,f:m_60-69,f:m_70-79,f:m_80-89
0,Amsterdam,105.60,104.80,104.79,104.42,98.92,98.37,99.67,99.88,97.86,...,77.62,60.19,40.74,98.62,99.77,98.25,101.10,97.83,91.37,70.77
1,Antwerp,104.58,104.60,104.00,106.72,104.67,100.04,99.91,102.48,103.73,...,75.02,60.64,43.87,102.15,101.17,103.95,103.50,97.38,86.86,68.89
2,Athens,104.66,104.65,106.73,113.61,109.55,97.28,92.17,97.77,95.78,...,64.99,59.08,63.55,103.15,95.28,92.76,83.80,80.18,77.33,62.68
3,Barcelona,105.71,106.76,107.62,108.08,106.35,100.08,97.15,95.13,99.01,...,67.37,55.19,38.11,103.01,96.06,99.75,96.25,87.69,80.13,61.77
4,Berlin,105.61,105.40,106.08,106.43,103.39,100.25,101.85,103.55,103.93,...,69.52,57.99,36.26,101.61,102.67,103.77,102.03,93.60,82.45,65.75


## Further reduction of the dataset

Having researched extensively to obtain the metrics above, we could only gather complete data for 15 of the 50 cities in the dataset. There were many cities with only partial data, and others completely missing all the features data for cities.

In order to develop a consistent prototype, we will focus on the 15 cities for which we have all the features. In future developments, we will increase the number of cities included in the dataset.

In [122]:
final_city_list = ['Paris','London','Rome','Berlin','Barcelona',
                   'Copenhagen','Amsterdam','Madrid','Vienna','Brussels',
                   'Dublin','Athens','Geneva','Antwerp','Manchester']

In [123]:
dataset_final = dataset[dataset['City'].isin(final_city_list)]

In [124]:
dataset_final[['Country','City']].value_counts()

Country         City      
France          Paris         54924
United Kingdom  London        48752
Italy           Rome          24416
Germany         Berlin        20114
Spain           Barcelona     17154
Denmark         Copenhagen    16024
Netherlands     Amsterdam     14969
Spain           Madrid        13031
Austria         Vienna         7738
Belgium         Brussels       5591
Ireland         Dublin         5199
Greece          Athens         4952
Switzerland     Geneva         2198
Belgium         Antwerp        1200
United Kingdom  Manchester      797
dtype: int64

In [125]:
dataset_final.shape

(237059, 41)

In [126]:
print('Before city name convergence, the first 50 cities included',np.round((first_50.sum()/282444),2)*100,'% of the listings in the dataset')
print('The top 50 cities after convergence included',np.round(first_50_bis.sum()/282444,2)*100,'% of the residences in the Europe region.')
print('After cleaning the dataset, the top 50 cities include',np.round(259415/282444,2)*100,'% of the residences in the Europe region.')
print('After selecting the final 15 cities in the dataset, the dataset includes',np.round(237059/282444,2)*100,'% of the residences in the Europe region.')

Before city name convergence, the first 50 cities included 91.0 % of the listings in the dataset
The top 50 cities after convergence included 95.0 % of the residences in the Europe region.
After cleaning the dataset, the top 50 cities include 92.0 % of the residences in the Europe region.
After selecting the final 15 cities in the dataset, the dataset includes 84.0 % of the residences in the Europe region.


In [127]:
# save the files into csv for EDA
dataset_final.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\dataset_final.csv')

In [128]:
#save of the cities features
economic.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\economic.csv')
crime.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\crime.csv')
pollution.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\pollution.csv')
amenities.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\amenities.csv')
hostility.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\hostility.csv')
tot_pop.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\population.csv')
female_pop.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\population female.csv')
male_pop.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\population male.csv')
female_rate.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\ratio female.csv')
male_rate.to_csv(r'C:\Users\34658\SPRINGBOARD\Airbnb Recommender System - Capstone Project\Notebooks\Data\residences\ratio male.csv')


## Summary of the work performed

In the Data Wrangling work performed we started with a world dataset with numerous Airbnb residences and features per residence. In particular around 500k residences.

Firstly, we filtered the dataset to include only information regarding cities in Europe region, but it was not clean as there were many cities with names written in different ways or in other languages, as well as buroughs within those cities named as separate cities. The size of the data was 91% of the original.

To solve this issue, we indiviually filtered country by country the top 15-40 cities (depending on how many residences appeared in the search) and merged the city names and buroughs to their cities, making a convergence. After this work, we subset in a different dataframe the top 50 cities and their residences by number count. The size of the data was 95% of the original.

After doing this, we split the dataset into different datasets, each with different groups of features, for example, features about the host, features about the residence or features about the price. We cleaned separately each file, imputing missing values following different criterias.

At last, we merged the different datasets into a single dataframe to have all the information regarding residences and hosts together and saved it. The size of the data was 92% of the original.

Besides this work, we added external information about the cities through different websites, the features included information about the cities characteristics from different perspectives, such as their pollution, crime rates, social culture, population or amenities.

Acknowledging that the cleaned dataset of residences had 50 cities, but we had only information about 15 cities in terms of city features, we further subset the residences dataset to include only cities for which we had features. The size of the dataset was 84% of the original.

As a result, we saved the final dataset as "dataset_final" and the city features datasets according to their content.